In [ ]:
import s4sim.hardware
c=s4sim.hardware.get_example()

In [ ]:
import s4sim

In [ ]:
git_hash = s4sim.__version__.split(".")[-1][1:]
git_hash

The git commit hash is the last section after the dot of this version id after removing the first character.

In [ ]:
c.data["telescopes"]["SAT1"]["fwhm"]

In [ ]:
c.data["telescopes"].keys()

In [ ]:
c.data["telescopes"]["SAT3"]["fwhm"].keys()

In [ ]:
c.data["telescopes"]["SAT3"].keys()

In [ ]:
c.data["bands"].keys()

In [ ]:
import numpy as np

In [ ]:
from collections import OrderedDict
from astropy import units as u
from astropy.table import QTable

In [ ]:
ipac_warning = ["Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')",
    f"Instrument model exported from `s4sim`, version {git_hash}"]

In [ ]:
c.data["telescopes"]["LAT0"]["fwhm"]

In [ ]:
s4 = []
for i, (band, v) in enumerate(c.data["bands"].items()):
    print("## " + band)
    print("Bandpass [GHz] low,center,high: ", v["low"], v["center"], v["high"])
    tel = band.split("_")[0][-3:]
    assert tel in ["SAT", "LAT"]
    fwhm = c.data["telescopes"][tel + "1"]["fwhm"][band]
    print("Beam [arcmin]: ", fwhm)
    d = OrderedDict(band=band)
    d["band"] = band
    d["telescope"] = tel
    if band.startswith("SP"):
        d["site"] = "Pole"
    elif band.startswith("CH"):
        d["site"] = "Chile"
    else:
        d["site"] = "Both"
    d["center_frequency"] = v["center"] * u.GHz
    d["fwhm"] = fwhm * u.arcmin
    # d["NET"] = v["NET"] * u.uK*u.s**.5
    d["nside"] = 4096 if tel =="LAT" else 512
    bandpass_table = QTable(dict(bandpass_frequency=np.linspace(v["low"], v["high"], 10) * u.GHz,
    bandpass_weight=np.ones(10, dtype=np.float)))
    bandpass_table.meta["comments"] = ipac_warning
    bandpass_table.write(f"../instrument_model/bandpass_{band}.tbl", format="ascii.ipac", overwrite=True)
    s4.append(d)

## Add reference to bandpasses

In [ ]:
for ch in s4:
    ch["bandpass_file"] = f"bandpass_{ch['band']}.tbl"

In [ ]:
len(s4)

In [ ]:
%ls ../instrument_model/band* | wc -l

# Create the astropy Table

In [ ]:
import astropy.units as u

In [ ]:
from astropy.table import QTable, Table

In [ ]:
t = QTable(rows=s4, masked=True, names=list(s4[1]))
t.add_index("band")

In [ ]:
t.meta["comments"] = ipac_warning

In [ ]:
t.sort(["telescope", "center_frequency"])

In [ ]:
t.write("../instrument_model/cmbs4_instrument_model.tbl", format="ascii.ipac", overwrite=True)

In [ ]:
c = QTable.read("../instrument_model/cmbs4_instrument_model.tbl", format="ascii.ipac" )

In [ ]:
c.add_index("band")

In [ ]:
c

In [ ]:
cat ../instrument_model/cmbs4_instrument_model.tbl

## Compare with the instrument model used to run the Chile LAT simulations

Check that center frequencies and fwhm are the same

In [ ]:
lat_instrument_model = QTable.read("../../202102_design_tool_run/instrument_model/cmbs4_instrument_model.tbl", format="ascii.ipac" )

In [ ]:
lat_instrument_model.add_index("band")

In [ ]:
lat_instrument_model = lat_instrument_model[lat_instrument_model["telescope"] == "LAT"]

In [ ]:
chlat_instrument_model = lat_instrument_model[["P" not in b for b in lat_instrument_model["band"]]]

In [ ]:
chlat_instrument_model

In [ ]:
chlat_new_instrument_model = c[c["site"] == "Chile"]

In [ ]:
chlat_new_instrument_model

In [ ]:
chlat_instrument_model["fwhm"] == chlat_new_instrument_model["fwhm"]

In [ ]:
chlat_instrument_model["center_frequency"] == chlat_new_instrument_model["center_frequency"]